In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib ipympl
plt.rcParams['figure.figsize'] = [7, 6]

import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.afw.cameraGeom as cameraGeom
import lsst.afw.cameraGeom.utils as cgUtils
import lsst.afw.math as afwMath

namp = 16  # number of amplifiers
ampColors = cm.rainbow(np.array(list(range(namp//2)) + list(range(2*namp//2, 3*namp//2)))/(3*namp//2))  # blues then reds

In [3]:
afwDisplay.setDefaultBackend("matplotlib")

In [4]:
butler = dafPersist.Butler(os.path.join("/project/rhl/Data/auxTel" if False else "/project/rhl/Data/ci_lsst/auxTel",
                                        "rerun/rhl/ci"))
camera = butler.get('camera')

In [5]:
if False:
    print(sorted(butler.queryMetadata("raw", ["dayObs", "seqnum"])))

In [6]:
if True:        # from CCS 
    dataId = dict(dayObs='2018-09-20', seqNum=37)
else:           # from OCS/CCS bridge
    dataId = dict(dayObs='2018-08-20', seqNum=15)
print(dataId)

{'dayObs': '2018-09-20', 'seqNum': 37}


In [7]:
if False:
    cgUtils.plotFocalPlane(camera)

In [8]:
from lsst.ip.isr import AssembleCcdTask
from lsst.ip.isr import IsrTask


config = AssembleCcdTask.ConfigClass()
config.doTrim = True

assembleTask = AssembleCcdTask(config=config)


config = IsrTask.ConfigClass()
config.overscanFitType = "AKIMA_SPLINE"
config.overscanOrder = 5
config.doBias = True
config.doDark = False
config.doFringe = False
config.doFlat = False
config.doLinearize = False
config.doDefect = False
config.doAddDistortionModel = False
config.doSaturationInterpolation = False
config.overscanNumLeadingColumnsToSkip = 20

isrTask = IsrTask(config=config)

In [20]:
#plt.close('all')
seqNum = 38
dataType = "postISRCCD"

exposure = butler.get(dataType, dataId, seqNum=seqNum)
title = dataType

if True:
    title += "*gain"
    for a in exposure.getDetector():
        exposure[a.getBBox()].maskedImage *= a.getGain()
    
bin = 4
if bin != 1:
    title += " %dx%d" % (bin, bin)
    exposure.maskedImage = afwMath.binImage(exposure.maskedImage, bin)

# ------

disp = afwDisplay.Display(2, reopenPlot=True)
disp.setImageColormap('gray')
   
if False:
    disp.scale('asinh', 'zscale')
else:
    if False:
        rng = np.percentile(exposure.image.array, [10, 90])
    else:
        rng = afwMath.makeStatistics(exposure.image, afwMath.MEDIAN).getValue() + 6*np.array([-1, 1])
    disp.scale('linear', *rng)

disp.mtv(exposure, title="%s, (%s, %s)" % (title, dataId["dayObs"], seqnum))
#disp.show_colorbar()
cgUtils.overlayCcdBoxes(exposure.getDetector(), binSize=bin, isTrimmed=True, display=disp)

Read all the data and store it in `exposures`

In [21]:
#del exposures
try:
    exposures
except NameError:
    exposures = {}

In [22]:
for seqNum in sorted(butler.queryMetadata("raw", ["seqNum"], dayObs=dataId["dayObs"])):
    dataId["seqNum"] = seqNum

    if True:
        dataType = 'postISRCCD'
    else:
        dataType = 'raw'        
            
    if not butler.datasetExists(dataType, dataId):
        continue

    exposure = butler.get(dataType, dataId)
        
    if True:
        title = "postISR"
    else:
        if False:
            title = "assembled"

            for a in exposure.getDetector():
                aim = exposure.image[a.getRawDataBBox()]
                aim -= np.median(exposure.image[a.getRawHorizontalOverscanBBox()].array)
            title += " - overscan"

            exposure = assembleTask.assembleCcd(exposure)
        else:
            title = "post-ISR"
            bias = butler.get('bias', dataId) if config.doBias else None

            exposure = isrTask.run(exposure, bias=bias).exposure

            for a in exposure.getDetector():
                exposure.maskedImage[a.getBBox()] *= a.getGain()
        
    exposures[dataId["seqNum"]] = exposure

{'dayObs': '2018-09-20', 'seqNum': 28}
{'dayObs': '2018-09-20', 'seqNum': 29}
{'dayObs': '2018-09-20', 'seqNum': 30}
{'dayObs': '2018-09-20', 'seqNum': 31}
{'dayObs': '2018-09-20', 'seqNum': 32}
{'dayObs': '2018-09-20', 'seqNum': 33}
{'dayObs': '2018-09-20', 'seqNum': 34}
{'dayObs': '2018-09-20', 'seqNum': 35}
{'dayObs': '2018-09-20', 'seqNum': 36}
{'dayObs': '2018-09-20', 'seqNum': 39}
{'dayObs': '2018-09-20', 'seqNum': 40}
{'dayObs': '2018-09-20', 'seqNum': 41}
{'dayObs': '2018-09-20', 'seqNum': 43}
{'dayObs': '2018-09-20', 'seqNum': 44}
{'dayObs': '2018-09-20', 'seqNum': 45}
{'dayObs': '2018-09-20', 'seqNum': 47}
{'dayObs': '2018-09-20', 'seqNum': 48}
{'dayObs': '2018-09-20', 'seqNum': 49}
{'dayObs': '2018-09-20', 'seqNum': 51}
{'dayObs': '2018-09-20', 'seqNum': 52}
{'dayObs': '2018-09-20', 'seqNum': 53}
{'dayObs': '2018-09-20', 'seqNum': 55}
{'dayObs': '2018-09-20', 'seqNum': 56}
{'dayObs': '2018-09-20', 'seqNum': 57}
{'dayObs': '2018-09-20', 'seqNum': 59}
{'dayObs': '2018-09-20', 

In [24]:
for s, exp in exposures.items():
    vi = exp.getInfo().getVisitInfo()
    d = vi.getDate()
    print(s, vi.getDate(), vi.getExposureTime())

37 DateTime("2018-09-21T05:44:38.777000000", TAI) 0.0
38 DateTime("2018-09-21T05:45:26.906000000", TAI) 1.0
42 DateTime("2018-09-21T05:48:32.625000000", TAI) 5.0
46 DateTime("2018-09-21T05:51:29.502000000", TAI) 9.0
50 DateTime("2018-09-21T05:55:58.001000000", TAI) 13.0
54 DateTime("2018-09-21T05:59:37.399000000", TAI) 17.0
58 DateTime("2018-09-21T06:03:56.545000000", TAI) 21.0
62 DateTime("2018-09-21T06:09:34.814000000", TAI) 25.0
66 DateTime("2018-09-21T06:14:16.453000000", TAI) 29.0
70 DateTime("2018-09-21T06:19:03.215000000", TAI) 0.0


In [26]:
s1 = 37
gain = {}
rn = {}

exp = exposures[s1]
#exp = exposure
for a in exp.getDetector():
    name = a.getName()
    stats = afwMath.makeStatistics(exp[a.getBBox()].image, afwMath.MEDIAN | afwMath.MEANCLIP | afwMath.STDEVCLIP)
    gain[name] = 1/stats.getValue(afwMath.MEANCLIP)
    rn[name] = stats.getValue(afwMath.STDEVCLIP)

med0 = np.min(list(gain.values()))
for n in gain:
    gain[n] /= med0
    
for a in exp.getDetector():
    name = a.getName()
    print("      %s : { gain : %.2f, readNoise : %5.2f }" % (name, gain[name], rn[name]*gain[name]))

      C10 : { gain : 0.43, readNoise :  4.97 }
      C11 : { gain : 0.18, readNoise :  2.04 }
      C12 : { gain : -0.14, readNoise : -1.61 }
      C13 : { gain : 0.07, readNoise :  0.87 }
      C14 : { gain : 0.11, readNoise :  1.26 }
      C15 : { gain : 0.06, readNoise :  0.74 }
      C16 : { gain : 0.17, readNoise :  1.95 }
      C17 : { gain : 0.21, readNoise :  3.02 }
      C07 : { gain : 0.16, readNoise :  1.78 }
      C06 : { gain : -1.40, readNoise : -15.95 }
      C05 : { gain : 1.00, readNoise : 12.15 }
      C04 : { gain : 0.23, readNoise :  2.68 }
      C03 : { gain : -0.12, readNoise : -1.43 }
      C02 : { gain : 0.10, readNoise :  1.21 }
      C01 : { gain : 0.33, readNoise :  3.90 }
      C00 : { gain : -0.55, readNoise : -6.58 }


In [28]:
disp = afwDisplay.Display(1, reopenPlot=True)

seqNum = 38
print(butler.get('raw_md', dataId, seqNum=seqNum).toDict())

bias = butler.get('raw', dataId, seqNum=seqNum)
rng = np.percentile(bias.image.array, [1, 99])

disp.scale('linear', *rng)
disp.mtv(bias, title="%s, %g - %g" % ("bias", *rng))
cgUtils.overlayCcdBoxes(bias.getDetector(), isTrimmed=True, display=disp)

{'BSCALE': 1.0, 'PC2_2Q': -1.0, 'PC2_1Q': 0.0, 'PC1_2Q': 0.0, 'PC1_1Q': -1.0, 'CTYPE2Q': 'RAFT_P', 'CTYPE1Q': 'RAFT_S', 'WCSNAMEQ': 'RAFT_SERPAR', 'CRVAL2B': 4001.0, 'CRVAL1B': 507.0, 'CRPIX2B': 0.0, 'CRPIX1B': 0.0, 'CDELT2B': 1.0, 'PC2_2B': -1.0, 'PC2_1B': 0.0, 'CTYPE2B': 'CCD_P', 'CTYPE1B': 'CCD_S', 'CRPIX1A': 0.0, 'CRVAL2C': 507.0, 'PC2_2A': 0.0, 'CDELT1A': 1.0, 'CTYPE2A': 'Seg_Y', 'PC2_1C': -1.0, 'BZERO': 0.0, 'DTM2_2': 1.0, 'DTV1': 513, 'CRVAL2Q': 4126.0, 'XTENSION': 'IMAGE', 'CDELT2A': 1.0, 'CRPIX2Q': 0.0, 'DATASEC': '[4:512,1:2000]', 'PC2_2C': 0.0, 'NAXIS2': 2048, 'STDEV': 35.974918661347026, 'DETSEC': '[509:1,1:2000]', 'PCOUNT': 0, 'DTM1_1': -1.0, 'PC1_1A': 0.0, 'DATASUM': '3727849697', 'CRVAL2F': 596.0, 'NAXIS1': 576, 'HEADVER': 1, 'CRVAL2R': 596.0, 'PC1_1B': -1.0, 'PC1_2F': -1.0, 'NAXIS': 2, 'DTV2': 0, 'PC2_1R': -1.0, 'CTYPE2R': 'RAFT_Y', 'DTM1_2': 0.0, 'BITPIX': 32, 'CRVAL2A': 507.0, 'PC1_1F': 0.0, 'GCOUNT': 1, 'CHECKSUM': 'aPYmaOWlaOWlaOWl', 'PC1_2B': 0.0, 'CRPIX2A': 0.0, '

RuntimeError: Tried to substitute exposureTime for darkTime but it is not available

In [ ]:
n = 10
plt.close(n)
plt.figure(n)

if False:
    exp = abias
    untrimmed = True
    title = "master bias"
else:
    seqnum = 608

    untrimmed = True
    subtractSignal = False and seqnum >= 607  # i.e. estimate and subtract the signal
    
    if untrimmed:
        from lsst.ip.isr import overscanCorrection
        raw = butler.get('raw', dataId, seqnum=seqnum)
        
        raw.maskedImage -= abias.maskedImage
        
        if True:
            doTrim = config.assembleCcd.doTrim
            try:
                config.assembleCcd.doTrim = False

                exposure = isrTask.run(raw, bias=abias).exposure
            except Exception as e:
                raise
            finally:
                config.assembleCcd.doTrim = doTrim
        else:
            for a in raw.getDetector():
                if True:
                    aim = raw.maskedImage[a.getRawDataBBox()]
                    aoscan = raw.maskedImage[a.getRawHorizontalOverscanBBox()]

                    overscanCorrection(aim, aoscan, config.overscanFitType, config.overscanOrder)
                elif True:
                    aim = raw.image[a.getRawBBox()]
                    aoscan = raw.image[a.getRawHorizontalOverscanBBox()]

                    aim -= np.median(aoscan.array)

                else:
                    aim = raw.image[a.getRawDataBBox()]
                    aoscan = raw.image[a.getRawHorizontalOverscanBBox()]

                    aim -= np.median(aoscan.array)

        exp = raw
    else:
        exp = exposures[seqnum]
    title = "dayObs=%s seqnum=%s" % (dataId["dayObs"], seqnum)

if not False:
    for i, a in enumerate(exp.getDetector()):
        aim = exp.image[a.getRawBBox() if untrimmed else a.getBBox()]

        if False:
            std = np.std(aim.array, axis=0)
        else:
            q1, q2, q3 = np.percentile(aim.array, [25, 50, 75], axis=0)
            std = 0.741*(q3 - q1)
                
        plt.plot(std, alpha=0.5, label=a.getName(), color=ampColors[i])

    if untrimmed:
        a = exp.getDetector()[0]
        for x in (a.getRawHorizontalOverscanBBox().getEndX(), a.getRawPrescanBBox().getBeginX()):
            plt.axvline(x, ls=':', color='black')

    plt.legend(loc='upper right', ncol=2).draggable()
    plt.ylim(-0.5, 16 if title == "master bias" else 30)
    plt.ylabel("Per-column std. deviation")
    plt.xlabel("Column")
    plt.title(title)

    n += 1
    plt.close(n)
    plt.figure(n)

avgmed = np.mean([a.getGain() for a in exp.getDetector()])

for i, a in enumerate(exp.getDetector()):
    if False and "C1" in a.getName():
        continue
    if untrimmed:
        bbox = a.getRawDataBBox()
        bbox.include(a.getRawPrescanBBox())
        bbox.include(a.getRawHorizontalOverscanBBox())
    else:
        bbox = a.getBBox()

    if False:
        med = np.median(exp.image[bbox].array, axis=0)
    elif False:    # afwMath is no better
        aim = exp.image[bbox]
        med = np.empty(aim.getWidth())
        for c in range(aim.getWidth()):
            med[c] = afwMath.makeStatistics(aim[c:c + 1, :, afwImage.LOCAL], afwMath.MEDIAN).getValue()
    else:
        sim = exp.image[bbox].array

        nblock = 10   # number of blocks in the row direction to average together
        h, w = sim.shape
        
        meds = np.empty((w, nblock))
        dr = h//nblock
        r0 = 0
        for ir in range(nblock):
            meds[:, ir] = np.median(sim[r0:r0 + dr, :], axis=0)
            r0 += dr
            if r0 + dr >= h:
                dr = h - dr - 1

        med = np.mean(meds, axis=1)

    if untrimmed and subtractSignal:
        x0 = a.getRawBBox().getBeginX()
        bbox = a.getRawDataBBox()
        data = med[bbox.getBeginX() - x0 : bbox.getEndX() - x0 + 1]
        data -= np.median(data[: 40])

    med *= a.getGain()*1.7/avgmed
    plt.plot(med, alpha=0.5, label=a.getName(), color=ampColors[i])
    
if untrimmed:
    pass
     
plt.axhline(0.0, ls=':', color='black')
if untrimmed:
    a = exp.getDetector()[0]
    for x in (a.getRawHorizontalOverscanBBox().getEndX(), a.getRawPrescanBBox().getBeginX()):
        plt.axvline(x, ls=':', color='black')

plt.legend(loc='upper left', ncol=2).draggable()
plt.xlim(-1, 80)
#plt.ylim(-100, 250)
plt.ylim(-10, 70)
plt.ylabel("Per-column median")
plt.xlabel("Column")
plt.title(title);

In [ ]:
rn = {}
gain = {}

sb1, sb2, s1, s2 = (600, 601, 607, 608) if False else (602, 603, 609, 611)

#texps = {}
for s in (sb1, sb2, s1, s2):
    if s not in texps:
        raw = butler.get('raw', dataId, seqnum=s)

        texps[s] = isrTask.run(raw, bias=bias).exposure

for a in exposures[s1].getDetector():
    name = a.getName()
    
    num =  texps[sb1][a.getBBox()].image.clone()
    num -= texps[sb2][a.getBBox()].image

    if False:
        num *= num
        rn[name] = np.sqrt(afwMath.makeStatistics(num, afwMath.MEANCLIP).getValue()/2)
    elif False:
        rn[name] = np.sqrt(afwMath.makeStatistics(num, afwMath.VARIANCECLIP).getValue()/2)
    else:
        q1, q2, q3 = np.percentile(num.array, [25, 50, 75])
        std = 0.741*(q3 - q1)

        rn[name] = std/np.sqrt(2)

    num =  texps[s1][a.getBBox()].image.clone()
    num -= texps[s2][a.getBBox()].image
    num *= num
    num -= rn[name]**2

    den =  texps[s1][a.getBBox()].image.clone()
    den += texps[s2][a.getBBox()].image
    
    num /= den
    
    gain[name] = 1/np.median(num.array)
    
for a in exposures[s1].getDetector():
    name = a.getName()
    print("      %s : { gain : %.2f, readNoise : %5.2f }" % (name, gain[name], rn[name]))

In [ ]:
exp = texps[s1].clone()
for a in exp.getDetector():
    name = a.getName()
    #exp.maskedImage[a.getBBox()] *= gain[name]

disp = afwDisplay.Display(3, backend="matplotlib", reopenPlot=True)

if False:
    disp.scale('asinh', 'zscale')
else:
    #rng = np.percentile(exp.image.array, [1, 99])
    disp.scale('linear', *rng)

disp.mtv(exp)
cgUtils.overlayCcdBoxes(exp.getDetector(), isTrimmed=True, display=disp)

In [ ]:
num =  texps[sb1].image.clone()
num -= texps[sb2].image
num *= num

den =  texps[s1].image.clone()
den += texps[s2].image

#num /= den

disp = afwDisplay.Display(3, backend="matplotlib", reopenPlot=True)

disp.scale('asinh', 'zscale')
disp.mtv(num)
cgUtils.overlayCcdBoxes(bias.getDetector(), isTrimmed=True, display=disp)

In [ ]:
disp = afwDisplay.Display(3, backend="matplotlib", reopenPlot=True)

disp.scale('asinh', 'zscale')
disp.mtv(raw)
cgUtils.overlayCcdBoxes(raw.getDetector(), isTrimmed=False, display=disp);

In [ ]:
n = 8
plt.close(n)
plt.figure(n)

plotTime = False

addLabels = True
t0 = None
for seqnum in exposures:
    exp = exposures[seqnum]

    vi = exp.getInfo().getVisitInfo()
    t = 3600*24*(vi.getDate().get() - 58350)
    if t0 is None:
        t0 = 1000*int(t/1000)
        
    if vi.getExposureTime() != 0.0:
        continue

    for i, a in enumerate(exp.getDetector()):
        aim = exp.image[a.getBBox()]

        plt.plot(t - t0 if plotTime else seqnum, np.median(aim.array), 'o', alpha=0.5, label=a.getName() if addLabels else None, color=ampColors[i])
     
    addLabels = False

plt.legend(loc='best', ncol=2)
plt.ylabel("Per-amp median")
plt.xlabel("time (seconds, arbitrary starttime)" if plotTime else "Seqnum")
plt.title("");

In [ ]:
disp = afwDisplay.Display(3, backend="matplotlib", reopenPlot=True)

bdataId = dict(dayObs='2018-08-23', seqnum=606)
biasExp = butler.get('raw', bdataId)

for a in biasExp.getDetector():
    aim = biasExp.image[a.getRawBBox()]
    overscanLevel = np.median(biasExp.image[a.getRawHorizontalOverscanBBox()].array)
    aim -= overscanLevel

rng = np.percentile(bias.image.array, [1, 99])

disp.scale('linear', *rng) # 'zscale')
disp.mtv(biasExp, title="%s, %s, %g - %g" % ("bias", str(bdataId), *rng))
cgUtils.overlayCcdBoxes(biasExp.getDetector(), isTrimmed=False, display=disp)

In [ ]:
import lsst.afw.table as afwTable
from lsst.meas.algorithms.detection import SourceDetectionTask

schema = afwTable.SourceTable.makeMinimalSchema()
config = SourceDetectionTask.ConfigClass()
config.thresholdValue = 15  # sigma
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)

tab = afwTable.SourceTable.make(schema)
subexp = exposure[exposure.getDetector()[11].getBBox()]
result = sourceDetectionTask.run(tab, subexp, sigma=2)
sources = result.sources

plt.close(2)
disp = afwDisplay.Display(2, backend="matplotlib", name='tmp')

disp.mtv(subexp)

brightest = None
for s in sources:
    peak = s.getFootprint().getPeaks()[0]
        
    pv = peak["peakValue"]
    if brightest is None or pv > brightest["peakValue"]:
        brightest = peak

xy = brightest.getF()
disp.dot('+', *xy)
disp.zoom(16, *xy)

In [ ]:
x, y = brightest.getI() - subexp.getXY0()

hsize = 40
x = int(x) + np.arange(-hsize, hsize + 1)
plt.close('all')
plt.plot(x + subexp.getX0(), subexp.image.array[y, x])
plt.axhline(0, color='black', ls=':')
plt.title(str(dataId));

In [ ]:
raw = butler.get('raw', dataId, seqnum=606)

if True:
    med0 = np.median(raw.image.array)
    for a in raw.getDetector():
        aim = raw.image[a.getRawHorizontalOverscanBBox()]
        med = np.median(aim.array) - med0
        #print(a.getName(), med + med0)
        raw.image[a.getRawBBox()] -= med
    
disp = afwDisplay.Display(2, reopenPlot=True)

disp.scale('asinh', 'zscale')
disp.mtv(raw, title="raw - median, %s" % (list(dataId.items())))
cgUtils.overlayCcdBoxes(raw.getDetector(), display=disp)

In [ ]:
names = []
parallelOverscan = []
serialOverscan   = []

raw = butler.get('raw', dataId)
for amp in raw.getDetector():
    names.append(amp.getName())
    # Serial overscan
    overscan = raw[amp.getRawHorizontalOverscanBBox()].image
    serialOverscan.append(np.median(overscan.array))
    # Parallel overscan (all on one line)
    parallelOverscan.append(np.median(raw[amp.getRawVerticalOverscanBBox()].image.array))
                
parallelOverscan = np.array(parallelOverscan)
serialOverscan   = np.array(serialOverscan)

In [ ]:
import matplotlib.cm as cm

colors = cm.rainbow(np.linspace(0, 1, len(names)))

min, max = np.percentile(parallelOverscan - serialOverscan, [0, 100])
patches = plt.hist((parallelOverscan - serialOverscan).reshape(1, len(names)), bins=np.linspace(min - 1, max + 1, 20),
                    color=colors, histtype='barstacked')[2]

plt.legend(patches, names, ncol=2)
plt.title("Serial overscan - parallel overscan (DN)")
plt.ylim(0, plt.ylim()[1] + 0.2);

In [ ]:
if True:
    did = dataId if True else dict(dayObs='2018-08-17', seqnum=556)
    f = butler.get("raw_filename", did)[0]
    md = butler.get("raw_md", did)
else:
    import lsst.afw.image as afwImage

    f = "/project/rhl/Data/auxTel/raw/Tucson/2018-08-16/AT-O-20180816-00008.fits"
    md = afwImage.readMetadata(f, hdu=0)
    
md = md.toDict()

print("%s\n" % f)
for k in sorted(md):
    print(k, md[k])

In [ ]:
import lsst.afw.image as afwImage

abias = afwImage.ExposureF("/project/rhl/Data/auxTel/rerun/rhl/calibTmp/bias/2018-08-24/bias-det000_2018-08-24.fits")  # bias created with oscan etc.
abias.setDetector(exposure.getDetector())

disp = afwDisplay.Display(4, backend="matplotlib", reopenPlot=True)

for a in abias.getDetector():
    aim = abias.image[a.getRawBBox()]
    overscanLevel = np.median(abias.image[a.getRawHorizontalOverscanBBox()].array)
    aim -= overscanLevel

rng = np.percentile(abias.image.array, [15, 85])

disp.scale('linear', *rng) # 'zscale')
disp.mtv(abias, title="%s, %s, %g - %g" % ("bias", "untrimmed", *rng))
cgUtils.overlayCcdBoxes(abias.getDetector(), isTrimmed=False, display=disp)
#disp.zoom(32, 4025, 2050)

In [ ]:
import lsst.obs.lsstCam.utils as lCamUtils
import lsst.afw.image as afwImage

bias = afwImage.ExposureF("/project/rhl/Data/auxTel/CALIB/bias/2018-08-20/bias-det000_2018-08-20.fits")

exp = lCamUtils.readRawFile("/project/rhl/Data/auxTel/raw/2018-08-20/02600015-det000.fits")

disp = afwDisplay.Display(1, "matplotlib", reopenPlot=True)
disp.scale('linear', 'zscale')
disp.mtv(isrTask.run(exp, bias=bias).exposure)

In [ ]:
import lsst.afw.image as afwImage
old = afwImage.ExposureF("/home/rlupton/LSST/pipe/tasks/tests/old.fits")
new = afwImage.ExposureF("/home/rlupton/LSST/pipe/tasks/tests/new.fits")

diff = new.clone()
diff.maskedImage -= old.maskedImage

exp = diff
exp = afwMath.binImage(exp.image, 1)

disp = afwDisplay.Display(4, backend="matplotlib", reopenPlot=True)
    
if True:
    rng = np.percentile(exp.array, [1, 99])
    disp.scale('linear', *rng)
else:
    disp.scale('asinh', 'zscale')

disp.mtv(exp)

In [ ]:
plt.close(10)
plt.figure(10)

plt.hist(diff.image.array.flatten(), bins=np.linspace(-0.10, 0.10, 50))
plt.title("New - Old");